In [1]:
from ipywidgets import widgets, interact
from IPython.display import display


def generate_random_point_on_elliptic_curve(E):
    while True:
        x = Integer(randrange(1, p))  # Случайное x в диапазоне [1, p-1]
        y_sqr = x**3 + E.a4() * x + E.a6()  # Уравнение кривой для определения y^2
        if y_sqr.is_square():
            y = y_sqr.sqrt()
            return E(x, y)
        
def generate_y_on_elliptic_curve(E, x):
    y_sqr = x**3 + E.a4() * x + E.a6()  # Уравнение кривой для определения y^2
    if y_sqr.is_square():
        y = y_sqr.sqrt()
        return y
    else:
        return -5

def CheckPoint(P, E): #CheckPoint(P, E): Эта функция проверяет, лежит ли точка P на эллиптической кривой E.
    x, y = P[0] ,P[1]

    if mod(y^2 + E.a1()*x*y + E.a3()*y,  p) == mod(x^3 + E.a2()*x^2  + E.a4()*x + E.a6(), p):
        return True
    else:
        return False

#JacobCoord(x, y, z, p, ToYac): Эта функция конвертирует точки между аффинными и Якобиановыми координатами.
#Если ToYac равно 1, 
#функция конвертирует из аффинных в Якобиановы координаты, иначе - наоборот.
def JacobCoord(x, y, z, p, ToYac):
    if ToYac == 1:
        if x == 0 and y == 1:
            return 1, 1, 0
        return mod(x/z^c, p), mod(y/z^d, p), mod(1/z, p)
    else:
        if x == 1 and y == 1 and z == 0:
            return 0, 1, 0
        return mod(x/z^c, p), mod(y/z^d, p) ,mod(1, p)
    
#double(X, Y, Z, A, p): Эта функция удваивает точку в якобиановых координатах. Функция возвращает координаты удвоенной точки.
def double(X, Y, Z, A, p):
    T1 = mod(X^2, p)
    T2 = mod(Y^2, p)
    T3 = mod(T2^2, p)
    C = mod(Z^2, p)
    
    S = mod(2*((X + T2)^2 - T1 - T3), p)
    
    M = mod(3*T1 + A*C^2, p)
    F = mod(M^2 - 2*S, p)
    
    X2 = F
    Y2 = mod(M*(S - F) - 8*T3, p)
    Z2 = mod((Y + Z)^2 - T2 - C, p)

    return X2, Y2, Z2

#addJacob(X1, Y1, Z1, X2, Y2, Z2, p): Эта функция добавляет 
#две точки в смешанных Якобианово-афинных координатах. Возвращает координаты результата.
def addJacob(X1, Y1, Z1, X2, Y2, Z2, p):
    if X2 == 0 and Y2 == 1:
        return X1, Y1, Z1
    
    T1 = mod(Z1^2, p)
    T2 = mod(Z2^2, p)
    U1 = mod(X1*T2, p)
    U2 = mod(X2*T1, p)
    
    S1 = mod(Y1*Z2*T2, p)
    S2 = mod(Y2*Z1*T1, p)
    H = mod(U2 - U1, p)
    I = mod((2*H)^2, p)
    
    J = mod(H*I, p)
    r = mod(2*(S2 - S1), p)
    V = mod(U1*I, p)
    
    X3 = mod(r^2 - J - 2*V, p)
    Y3 = mod(r*(V - X3) - 2*S1*J, p)
    Z3 = mod(((Z1 + Z2)^2 - T1 - T2)*H, p)

    return (X3, Y3, Z3)

#addPoints(P, Q, E): Эта функция принимает две точки P и Q в аффинных координатах, преобразует их в якобиановы координаты,
#добавляет их вместе с использованием функции addJacob, а затем преобразует результат обратно в аффинные координаты.
def addPoints(P, Q, E):
    z = randint(1, p-1)

    if P[0] == 0 and P[1] == 1:
        Xp, Yp, Zp = 1, 1, 0
    else:
        Xp, Yp, Zp = JacobCoord(P[0], P[1], z, p, 1)

    Xr, Yr, Zr = addJacob(Xp, Yp, Zp, Q[0], Q[1], 1, p)
    Xa, Ya, Za = JacobCoord(Xr, Yr, Zr, p, 0)

    return E(Xa, Ya, Za)

#doublePoint(P, E): Эта функция принимает точку P в аффинных координатах, преобразует ее в якобиановы координаты,
#удваивает ее с использованием функции double, а затем преобразует результат обратно в аффинные координаты.
def doublePoint(P, E):
    z = randint(1, p-1)
    
    Xp, Yp, Zp = JacobCoord(P[0], P[1], z, p, 1)
    Xd, Yd, Zd = double(Xp, Yp, Zp, A, p)
    Xa, Ya, Za = JacobCoord(Xd, Yd, Zd, p, 0)
    
    return E(Xa, Ya, Za)

#binaryMult(P, E, k): Эта функция выполняет двоичное
#умножение точки P на число k с использованием сложения и удвоения точек эллиптической кривой.
def binaryMult(P, E, k):
    Q = E(0, 1, 0)
    k_bin = k.binary()
    t = len(k_bin)
    i = t - 1

    while i >= 0 :
        if k_bin[i] == '1':
            Q = addPoints(P, Q, E)
        P = doublePoint(P, E)
        i -= 1
    return Q

#task(E): Эта функция выполняет серию операций, 
#демонстрирующих работу функций сложения, удвоения и бинарного умножения на примерах.
def task(E):
    print('\n p=',p)
    G = E(Xg, Yg)
    
    print ("\nG =", G)
    print()
    print ("\n2G (Sage) =", 2*G)
    print()
    print ("\n2G =", binaryMult(G, E, 2))
    print()
    
    G1 = 4004*G
    G1_test = binaryMult(G, E, 4004)
   
   
    G2 = 4321000000000000000000000000001234*G
    G2_test = binaryMult(G, E, 4321000000000000000000000000001234)
    
    
    G3 = G1 + G2
    G3_test = addPoints(G1, G2, E)
    print ("\n4004G1 + 4321000000000000000000000000001234G (Sage) =", G3)
    print()
    print ("\n4004G1 + 4321000000000000000000000000001234G =", G3_test)

    
Z = 1
c = 2
d = 3
p = None
A = None
B = None
Xg = None
Yg = None
X11 = None
Y11 = None
X22 = None
Y22 = None
 
#p = int(input("p = "))
#A = int(input("A = "))
#B = int(input("B = "))



        
def run_lab_variant(b):
    global p, A, B,Xg, Yg, E
    p = 115792089237316193816632940749697632408932724669461758699102671622853342616127
    A = 10001090682862736796505780343107022911835438229291751861888571869223050424594
    B = 45264756867680555803214833811970559410867867042681754140959938453766481155105    
    E = EllipticCurve(GF(p), [A, B])
    print('\n')
    print(E)
    print()

    #random_point1 = E.random_point()
    random_point1 = generate_random_point_on_elliptic_curve(E)
    random_point2 = binaryMult(random_point1, E, 2)
    random_point3 = binaryMult(random_point1, E, 3)

    print("P:", random_point1)

    if (CheckPoint(random_point1, E)):
        print("The point P lies on the curve\n")
    else:
        print("The point P is not lies on the curve\n")

    print("2P:", random_point2)
    print()
    print("3P:", random_point3)
    print()

    #Xg = int(input("Xg = "))
    #Yg = int(input("Yg = "))

    Xg = 34131522064967883044818697086599304935071739419701303653954060188761196377435
    #yy = generate_y_on_elliptic_curve(E, Xg)
    #print("yy = ", yy)
    Yg = 22161743160862324650134218095423264926581638655388631138889754360346138331114
    #print("Yg = ", Yg)
    #W = 4004
    #W2 = 4321000000000000000000000000001234
    if (CheckPoint(E(Xg, Yg), E)):
        print("The point P lies on the curve\n")
    else:
        print("The point P is not lies on the curve\n")
    

    task(E)

    
def input_function(b):
    global p, A, B, Xg, Yg, E
    
    p = p_input.value
    A = A_input.value
    B = B_input.value

    E = EllipticCurve(GF(p), [A, B])
    print('\n')
    print(E)
    print("Возможная точка: ", generate_random_point_on_elliptic_curve(E))
    print("Возможная точка: ", generate_random_point_on_elliptic_curve(E))
    print("Возможная точка: ", generate_random_point_on_elliptic_curve(E))
    print("Возможная точка: ", generate_random_point_on_elliptic_curve(E))
    
        
def input_function2(b):
    global p, A, B, X11, Y11, E
    X11 = Xg_input.value
    Y11 = generate_y_on_elliptic_curve(E, X11)
    
    if Y11 == -5:
        print("Такой точки нет")
    else:
        print("X1 = ", X11)
        print("Y1 = ", Y11)
        
def input_function3(b):
    global p, A, B, X22, Y22
    X22 = Xg2_input.value
    Y22 = generate_y_on_elliptic_curve(E, X22)
    
    if Y22 == -5:
        print("Такой точки нет")
    else:
        print("X2 = ", X22)
        print("Y2 = ", Y22)
        
def task_function(b):
    G1 = E(X11, Y11)
    G2 = E(X22, Y22)
    print('\np = ', p)
    print('A = ', A)
    print('B = ', B)
    print ("G1 =", G1)
    print ("G2 =", G2)
    print()
    print ("\nSage. 2*G1 = ", 2*G1)
    print ("2*G1 =", binaryMult(G1, E, 2))
    print()
    G3 = G1 + G2
    G3_test = addPoints(G1, G2, E)
    print ("\nSage. G1 + G2 = ", G3)
    print ("G1 + G2 =", G3_test)
    
def generate_random_X1(b):
    random_X1 = generate_random_point_on_elliptic_curve(E)
    Xg_input.value = random_X1[0]
    
def generate_random_X2(b):
    random_X2 = generate_random_point_on_elliptic_curve(E)
    Xg2_input.value = random_X2[0]

p_input = widgets.IntText(description='p = ')
A_input = widgets.IntText(description='A = ')
B_input = widgets.IntText(description='B = ')
Xg_input = widgets.IntText(description='X1 = ')
Xg2_input = widgets.IntText(description='X2 = ')

# Создаем кнопку для выполнения ввода
input_button = widgets.Button(description="Ввести значения (p, A, B)", layout=widgets.Layout(width="300px", height="50px"))
input_button.on_click(input_function)

input_button2 = widgets.Button(description="Ввести значение (X1)", layout=widgets.Layout(width="300px", height="50px"))
input_button2.on_click(input_function2)

input_button3 = widgets.Button(description="Ввести значение (X2)", layout=widgets.Layout(width="300px", height="50px"))
input_button3.on_click(input_function3)

task_button = widgets.Button(description="Произвести вычисления с точками", layout=widgets.Layout(width="300px", height="50px"))
task_button.on_click(task_function)

generate_button = widgets.Button(description='Сгенерировать X1', layout=widgets.Layout(width="300px", height="50px"))
generate_button.on_click(generate_random_X1)  

generate_button2 = widgets.Button(description='Сгенерировать X2', layout=widgets.Layout(width="300px", height="50px"))
generate_button2.on_click(generate_random_X2)  
    
def use_random_point(b):
    random_point1 = generate_random_point_on_elliptic_curve(E)
    random_point2 = binaryMult(random_point1, E, 2)
    random_point3 = binaryMult(random_point1, E, 3)

    print("P:", random_point1)

    if (CheckPoint(random_point1, E)):
        print("The point P lies on the curve\n")
    else:
        print("The point P is not lies on the curve\n")

    print("2P:", random_point2)
    print()
    print("3P:", random_point3)
    print()


    
    
# Создаем кнопки
button_lab = widgets.Button(description="Запустить вариант из лабораторной", layout=widgets.Layout(width="300px", height="50px"))
button_random = widgets.Button(description="Умножение случайной точки", layout=widgets.Layout(width="300px", height="50px"))

# Назначаем обработчики событий на кнопки
button_lab.on_click(run_lab_variant)
button_random.on_click(use_random_point)

# Отображаем кнопки
display(button_lab, button_random, p_input, A_input, B_input, input_button, Xg_input,generate_button, input_button2,Xg2_input,generate_button2, input_button3,task_button)



#p = 17 A = 2 B = 2
#p = 11 A = 2 B = 7
#p = 31 A = 6 B = 9

#P1 = (16, 4) P2 = (7, 11)



Button(description='Запустить вариант из лабораторной', layout=Layout(height='50px', width='300px'), style=But…

Button(description='Умножение случайной точки', layout=Layout(height='50px', width='300px'), style=ButtonStyle…

IntText(value=0, description='p = ')

IntText(value=0, description='A = ')

IntText(value=0, description='B = ')

...

Button(description='Ввести значение (X1)', layout=Layout(height='50px', width='300px'), style=ButtonStyle())

IntText(value=0, description='X2 = ')

Button(description='Сгенерировать X2', layout=Layout(height='50px', width='300px'), style=ButtonStyle())

Button(description='Ввести значение (X2)', layout=Layout(height='50px', width='300px'), style=ButtonStyle())

Button(description='Произвести вычисления с точками', layout=Layout(height='50px', width='300px'), style=Butto…